### Import Libraries and Data

In [35]:
import pandas as pd
import numpy as np
from itertools import product
import os


# URLs
base = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/"
df_umsatz = pd.read_csv(base + "umsatzdaten_gekuerzt.csv")
df_wetter = pd.read_csv(base + "wetter.csv")
df_kiwo = pd.read_csv(base + "kiwo.csv")

# Datum vereinheitlichen
for df in [df_umsatz, df_wetter, df_kiwo]:
    df["Datum"] = pd.to_datetime(df["Datum"])
    
    
# KiWo binär markieren
df_kiwo["KiWo"] = 1

# Vollständige Datum-Warengruppe-Kombination
alle_daten = pd.date_range(start="2013-07-01", end="2019-07-31", freq="D")
warengruppen = [1, 2, 3, 4, 5, 6]
voll_kombis = pd.DataFrame(list(product(alle_daten, warengruppen)), columns=["Datum", "Warengruppe"])

# Merge Wetter nur auf Datum (jeder Tag bekommt Wetterdaten)
df = voll_kombis.merge(df_wetter, on="Datum", how="left")

# Merge KiWo auf Datum (KiWo für jeden Tag)
df = df.merge(df_kiwo[["Datum", "KiWo"]], on="Datum", how="left")
df["KiWo"] = df["KiWo"].fillna(0).astype(int)

# Merge Umsatz auf Datum+Warengruppe (Umsatz nur, wenn vorhanden)
df = df.merge(df_umsatz, on=["Datum", "Warengruppe"], how="left")


# ID generieren (YYMMDDX)
df["id"] = df.apply(lambda row: int(row["Datum"].strftime("%y%m%d") + str(row["Warengruppe"])), axis=1)

df.head()
df.tail()


,Datum,Warengruppe,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KiWo,id,Umsatz
13327,2019-07-31,2,6.0,20.45,7.0,61.0,0,1907312,NaN
13328,2019-07-31,3,6.0,20.45,7.0,61.0,0,1907313,NaN
13329,2019-07-31,4,6.0,20.45,7.0,61.0,0,1907314,NaN
13330,2019-07-31,5,6.0,20.45,7.0,61.0,0,1907315,NaN
13331,2019-07-31,6,6.0,20.45,7.0,61.0,0,1907316,NaN


### Data Preparation

In [ ]:
# --- 1. Kategorische Features definieren ---
categorical_features = ['Warengruppe', 'Wettercode']

# Datentyp prüfen & eindeutige Werte
print(df[categorical_features].dtypes)
print("Unique Values:\n", df[categorical_features].apply(lambda x: x.unique()))

#  Datenqualität untersuchen 
print("Fehlende Werte:\n", df.isnull().sum())

print("\nUnplausible Werte prüfen:")
print("Temperatur < -50 oder > 60:", df[(df['Temperatur'] < -50) | (df['Temperatur'] > 60)].shape[0])
print("Windgeschwindigkeit < 0:", df[df['Windgeschwindigkeit'] < 0].shape[0])
print("Bewoelkung < 0:", df[df['Bewoelkung'] < 0].shape[0])
print("Umsatz < 0:", df[df['Umsatz'] < 0].shape[0])



# --- 2. Als Kategorien casten ---
for col in categorical_features:
    df[col] = df[col].astype('category')

# --- 3. One-hot-Encoding ---
features = pd.get_dummies(df[categorical_features], drop_first=False, dtype=int)

# --- 4. Numerische Spalten hinzufügen ---
features['Temperatur'] = df['Temperatur']
features['Bewoelkung'] = df['Bewoelkung']
features['Windgeschwindigkeit'] = df['Windgeschwindigkeit']
features['KiWo'] = df['KiWo']
###id hinzufügen?
features['id'] = df['id']
features['Datum'] = df['Datum']

## für Sortierung
features['Warengruppe'] = df['Warengruppe']


# --- 5. Zielvariable hinzufügen ---
prepared_data = pd.concat([df[['Umsatz']], features], axis=1)

""" # --- 6. Fehlende oder unplausible Werte entfernen ---
# Umsatz muss positiv sein; Temperatur und Wind realistisch
prepared_data = prepared_data[
    (prepared_data['Temperatur'] > -50) &
    (prepared_data['Temperatur'] < 60) 
    (prepared_data['Windgeschwindigkeit'] >= 0) &
    (prepared_data['Bewoelkung'] >= 0) &
    (prepared_data['Umsatz'] >= 0)
]
"""
# Entfernen von Zeilen mit fehlenden Werten, aber nicht für den test data, da wir sie später für die Vorhersage brauchen könnten.
# --- 7. Alle Zeilen mit fehlenden Werten löschen ---
#prepared_data = prepared_data.dropna()


# --- 8. Ausgabe ---
print("Prepared data shape:", prepared_data.shape)
prepared_data.head()
prepared_data.tail()


Warengruppe      int64
Wettercode     float64
dtype: object
Unique Values:
 Warengruppe                                   [1, 2, 3, 4, 5, 6]
Wettercode     [20.0, nan, 61.0, 5.0, 21.0, 65.0, 63.0, 95.0,...
dtype: object
Fehlende Werte:
 Datum                     0
Warengruppe               0
Bewoelkung              156
Temperatur               96
Windgeschwindigkeit      96
Wettercode             3204
KiWo                      0
id                        0
Umsatz                 3998
dtype: int64

Unplausible Werte prüfen:
Temperatur < -50 oder > 60: 0
Windgeschwindigkeit < 0: 0
Bewoelkung < 0: 0
Umsatz < 0: 0
Prepared data shape: (13332, 38)


,Umsatz,Warengruppe_1,Warengruppe_2,Warengruppe_3,Warengruppe_4,Warengruppe_5,Warengruppe_6,Wettercode_0.0,Wettercode_3.0,Wettercode_5.0,...,Wettercode_77.0,Wettercode_79.0,Wettercode_95.0,Temperatur,Bewoelkung,Windgeschwindigkeit,KiWo,id,Datum,Warengruppe
13327,NaN,0,1,0,0,0,0,0,0,0,...,0,0,0,20.45,6.0,7.0,0,1907312,2019-07-31,2
13328,NaN,0,0,1,0,0,0,0,0,0,...,0,0,0,20.45,6.0,7.0,0,1907313,2019-07-31,3
13329,NaN,0,0,0,1,0,0,0,0,0,...,0,0,0,20.45,6.0,7.0,0,1907314,2019-07-31,4
13330,NaN,0,0,0,0,1,0,0,0,0,...,0,0,0,20.45,6.0,7.0,0,1907315,2019-07-31,5
13331,NaN,0,0,0,0,0,1,0,0,0,...,0,0,0,20.45,6.0,7.0,0,1907316,2019-07-31,6


### Selction of Training, Validation and Test Data

In [ ]:




# Zeitliche Splits
train_mask = (prepared_data["Datum"] >= "2013-07-01") & (prepared_data["Datum"] <= "2017-07-31")
val_mask   = (prepared_data["Datum"] >= "2017-08-01") & (prepared_data["Datum"] <= "2018-07-31")
test_mask  = (prepared_data["Datum"] >= "2018-08-01") & (prepared_data["Datum"] <= "2019-07-31")

df_train = prepared_data[train_mask].copy()
df_val   = prepared_data[val_mask].copy()
df_test  = prepared_data[test_mask].copy()

df_train = df_train.dropna()
df_val = df_val.dropna()

print(df_train.columns)

# Sortierung
df_train = df_train.sort_values(by=["Warengruppe", "Datum"]).reset_index(drop=True)
df_val   = df_val.sort_values(by=["Warengruppe", "Datum"]).reset_index(drop=True)
df_test  = df_test.sort_values(by=["Warengruppe", "Datum"]).reset_index(drop=True)


#Begrenzung: Nur Warengruppe 6 auf 01.11.2018 bis 28.12.2018
df_test = df_test[
    ~((df_test["Warengruppe"] == 6) & (
        (df_test["Datum"] < "2018-11-01") | (df_test["Datum"] > "2018-12-28")
    ))
].copy()

# Eingabedaten (X) = alle Spalten außer 'Umsatz'
training_features = df_train.drop('Umsatz', axis=1)
training_features = training_features.drop('id', axis=1)    
training_features = training_features.drop('Datum', axis=1)
# Validation und Test Features

validation_features = df_val.drop('Umsatz', axis=1)
validation_features = validation_features.drop('id', axis=1)
validation_features = validation_features.drop('Datum', axis=1)
test_features = df_test.drop('Umsatz', axis=1)
test_features = test_features.drop('id', axis=1)
test_features = test_features.drop('Datum', axis=1)

# Zielwert (y) = nur 'Umsatz'-Spalte
training_labels = df_train[['Umsatz']]
validation_labels = df_val[['Umsatz']]
test_labels = df_test[['Umsatz']]



# Submission vorbereiten
df_submission = df_test[["id"]].copy()
df_submission["Umsatz"] = 0

# Kontrolle
print("Train:", df_train.shape)
print("Val:", df_val.shape)
print("Test:", df_test.shape)
print("\nNaNs in Train:\n", df_train.isna().sum())
print("\nNaNs in Val:\n", df_val.isna().sum())
print("\nNaNs in Test:\n", df_test.isna().sum())


# Print dimensions of the dataframes
print("Training features dimensions:", training_features.shape)
print("Validation features dimensions:", validation_features.shape)
print("Test features dimensions:", test_features.shape)
print()
print("Training labels dimensions:", training_labels.shape)
print("Validation labels dimensions:", validation_labels.shape)
print("Test labels dimensions:", test_labels.shape)


Index(['Umsatz', 'Warengruppe_1', 'Warengruppe_2', 'Warengruppe_3',
       'Warengruppe_4', 'Warengruppe_5', 'Warengruppe_6', 'Wettercode_0.0',
       'Wettercode_3.0', 'Wettercode_5.0', 'Wettercode_10.0',
       'Wettercode_17.0', 'Wettercode_20.0', 'Wettercode_21.0',
       'Wettercode_22.0', 'Wettercode_28.0', 'Wettercode_45.0',
       'Wettercode_49.0', 'Wettercode_53.0', 'Wettercode_55.0',
       'Wettercode_61.0', 'Wettercode_63.0', 'Wettercode_65.0',
       'Wettercode_68.0', 'Wettercode_69.0', 'Wettercode_71.0',
       'Wettercode_73.0', 'Wettercode_75.0', 'Wettercode_77.0',
       'Wettercode_79.0', 'Wettercode_95.0', 'Temperatur', 'Bewoelkung',
       'Windgeschwindigkeit', 'KiWo', 'id', 'Datum', 'Warengruppe'],
      dtype='object')
Train: (7487, 38)
Val: (1777, 38)
Test: (1883, 38)

NaNs in Train:
 Umsatz                 0
Warengruppe_1          0
Warengruppe_2          0
Warengruppe_3          0
Warengruppe_4          0
Warengruppe_5          0
Warengruppe_6          0
Wet

### Data Export

In [38]:
# Create subdirectory for the pickle files
subdirectory = "pickle_data"
os.makedirs(subdirectory, exist_ok=True)

# Export of the prepared data to subdirectory as pickle files
training_features.to_pickle(f"{subdirectory}/training_features.pkl")
validation_features.to_pickle(f"{subdirectory}/validation_features.pkl")
test_features.to_pickle(f"{subdirectory}/test_features.pkl")
training_labels.to_pickle(f"{subdirectory}/training_labels.pkl")
validation_labels.to_pickle(f"{subdirectory}/validation_labels.pkl")
test_labels.to_pickle(f"{subdirectory}/test_labels.pkl")